In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary
from models.resnet import ResNet, model_parameters  # Import ResNet từ mã của bạn


torch.Size([1, 256, 56, 56])


In [3]:
# Kiểm tra GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Biến đổi dữ liệu (chuẩn hóa và augment)
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  
    transforms.RandomCrop(32, padding=4),  
    transforms.ToTensor(),  
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  
])

# Load dataset CIFAR-10
trainset = torchvision.datasets.CIFAR10(root='./datasets', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./datasets', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

# Danh sách nhãn
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


100%|██████████| 170M/170M [00:21<00:00, 7.87MB/s] 


Extracting ./datasets\cifar-10-python.tar.gz to ./datasets
Files already downloaded and verified


In [4]:
# Chọn kiến trúc ResNet (vd: resnet18, resnet34, resnet50, ...)
architecture = 'resnet50'  # Hoặc 'resnet34', 'resnet50', 'resnet101'
resnet_variant = model_parameters[architecture]

# Tạo mô hình từ mã của bạn
model = ResNet(resnet_variant, in_channels=3, num_classes=10).to(device)

# Hiển thị cấu trúc mô hình
summary(model, (3, 32, 32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           9,408
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
         MaxPool2d-4           [-1, 64, 16, 16]               0
            Conv2d-5           [-1, 64, 16, 16]           4,096
       BatchNorm2d-6           [-1, 64, 16, 16]             128
              ReLU-7           [-1, 64, 16, 16]               0
            Conv2d-8           [-1, 64, 16, 16]          36,864
       BatchNorm2d-9           [-1, 64, 16, 16]             128
             ReLU-10           [-1, 64, 16, 16]               0
           Conv2d-11          [-1, 256, 16, 16]          16,384
      BatchNorm2d-12          [-1, 256, 16, 16]             512
           Conv2d-13          [-1, 256, 16, 16]          16,384
      BatchNorm2d-14          [-1, 256,

In [5]:
criterion = nn.CrossEntropyLoss()  # Hàm mất mát CrossEntropy cho bài toán phân loại
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  # Sử dụng Adam optimizer


In [6]:
num_epochs = 20  # Số epoch bạn muốn chạy

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # Xóa gradient trước đó
        outputs = model(inputs)  # Dự đoán đầu ra
        loss = criterion(outputs, labels)  # Tính loss
        loss.backward()  # Lan truyền ngược
        optimizer.step()  # Cập nhật trọng số

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}")


Epoch [1/20], Loss: 1.8066
Epoch [2/20], Loss: 1.3793
Epoch [3/20], Loss: 1.1250
Epoch [4/20], Loss: 0.9577
Epoch [5/20], Loss: 0.8434
Epoch [6/20], Loss: 0.7591
Epoch [7/20], Loss: 0.7038
Epoch [8/20], Loss: 0.6662
Epoch [9/20], Loss: 0.6200
Epoch [10/20], Loss: 0.5927
Epoch [11/20], Loss: 0.5576
Epoch [12/20], Loss: 0.5356
Epoch [13/20], Loss: 0.5234
Epoch [14/20], Loss: 0.4969
Epoch [15/20], Loss: 0.4762
Epoch [16/20], Loss: 0.4651
Epoch [17/20], Loss: 0.4440
Epoch [18/20], Loss: 0.4294
Epoch [19/20], Loss: 0.4145
Epoch [20/20], Loss: 0.4068


In [7]:
model.eval()
correct = 0
total = 0

with torch.no_grad():  # Không tính gradient khi đánh giá
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 82.40%
